In [1]:
from __future__ import absolute_import, division, print_function
import argparse
import os
import re
from collections import defaultdict
import glob
import time
import pathlib
import imageio
import sys
import numpy as np

import imageio

from sklearn.metrics import pairwise_distances

In [2]:
import tensorflow as tf
from torchvision.datasets import CIFAR10, STL10
import torch
import torchvision.utils as vutils
import torch.utils.data as utils
import visdom
from torchvision import transforms
from GAN_training.models import resnet, resnet_extra, resnet_48
from tqdm import tqdm, tqdm_notebook

import data

In [3]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipympl

In [4]:
class optclass:
    workaround = True
    
xp = '/scratch0/ilya/locDoc/ACGAN/experiments/'
    
opt = optclass()
optdict = {
    #'outf': '/scratch0/ilya/locDoc/ACGAN/experiments/yogesh_acgan_0p2',
    'outf': xp+'marygan-stl-48-miyato-hyp-lrGp4-auxp4',
    'netG': xp+'marygan-stl-48-miyato-hyp-lrGp4-auxp4/netG_iter_069999.pth',
    'marygan': True,
    'imageSize': 48,
    'data_root': '/scratch0/ilya/locDoc/data/stl10',
    'dataset': 'cifar',
    'dev_batch_size': 100,
    'size_labeled_data': 4000,
    'train_batch_size': 128,
    'train_batch_size_2': 100,
    'cifar_fname': '/scratch0/ilya/locDoc/data/cifar10/fid_is_scores.npz',
    'nz': 128,
    'GAN_nz': 128,
    'ngpu': 1,
    'nc':3,
    'num_classes': 10
}
for k, v in optdict.items():
    setattr(opt, k, v)

### Three celebA networks

In [842]:
opt.outf = xp+'celeb_cpy/celeba_vanillagan'
opt.netG = opt.outf+'/netG_iter_129999.pth' # 4.130121811844333
opt.marygan = False
opt.imageSize = 64

In [867]:
opt.outf = xp+'celeb_cpy/celeba5c_marygan'
opt.netG = opt.outf+'/netG_iter_129999.pth' # 3.6509132644800673
opt.marygan = True
opt.imageSize = 64

In [850]:
opt.outf = xp+'celeb_cpy/celeba5c_acgan'
opt.netG = opt.outf+'/netG_iter_119999.pth' # 5.074366134380284 
opt.marygan = False
opt.imageSize = 64

### Three STL networks

In [666]:
opt.outf = xp+'marygan-stl-48-miyato-hyp-lrGp4-auxp4'
opt.netG = opt.outf+'/netG_iter_069999.pth'
opt.marygan = True
opt.imageSize = 48

In [656]:
opt.outf = xp+'vanillagan-stl-48-miyato-hyp-lrGp4'
opt.netG = opt.outf+'/netG_iter_089999.pth'
opt.marygan = False
opt.imageSize = 48

In [639]:
opt.outf = xp+'yogesh-acgan-stl-48-miyato-hyp-lrGp4'
opt.netG = opt.outf+'/netG_iter_089999.pth'
opt.marygan = False
opt.imageSize = 48

### Three CIFAR networks

In [84]:
opt.outf = xp+'yogesh_marygan_0p2'
# opt.netG = opt.outf+'/netG_iter_399999.pth'
opt.netG = opt.outf+'/netG_iter_399999.pth'
opt.marygan = True
opt.imageSize = 32 

In [102]:
opt.outf = xp+'yogesh_acgan_0p2'
opt.netG = opt.outf+'/netG_iter_449999.pth'
opt.marygan = False
opt.imageSize = 32

In [114]:
opt.outf = xp+'yogesh_vanillagan_cifar'
opt.netG = opt.outf+'/netG_iter_489999.pth' # 14.711713683439996 8.034166
opt.marygan = False
opt.imageSize = 32

## Create images with the GAN

In [115]:
if opt.netG == '':
    netGfiles = glob.glob(os.path.join(opt.outf, 'netG_iter_*.pth'))
    netGfiles.sort(key = lambda s: int(s.split('_')[-1].split('.')[0]))
    opt.netG = netGfiles[-1]
    print(opt.netG)

In [116]:
if opt.imageSize == 32:
    netG = resnet.Generator(opt)
elif opt.imageSize == 64:
    netG = resnet_extra.Generator(opt)
elif opt.imageSize == 48:
    netG = resnet_48.Generator(opt)
netG.load_state_dict(torch.load(opt.netG))
netG = netG.cuda()

/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:112: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.dense.weight.data, 1.)
/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:113: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.final.weight.data, 1.)
/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.conv1.weight.data, 1.)
/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:16: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.conv2.weight.data, 1.)


In [117]:
batch_size = opt.train_batch_size
nz = opt.nz
noise = torch.FloatTensor(opt.train_batch_size, nz)
noise = noise.cuda()
num_classes = 10
klass_picked = None

# create images
n_used_imgs = 1000
n_gen_imgs = ((n_used_imgs // opt.train_batch_size) + 1) * opt.train_batch_size
x = np.empty((n_gen_imgs,3,opt.imageSize,opt.imageSize), dtype=np.uint8)
save_noise = np.empty((n_gen_imgs,128))
# create a bunch of GAN images
for l in  tqdm_notebook(range((n_used_imgs // opt.train_batch_size) + 1),desc='Generating'):
    start = l * opt.train_batch_size
    end = start + opt.train_batch_size
    noise.data.resize_(batch_size, nz).normal_(0, 1)
    #label = np.random.randint(0, num_classes, batch_size)
    if klass_picked is None:
        label = np.random.randint(0, num_classes, batch_size)
    else:
        label = np.ones((batch_size,),dtype=int)*klass_picked
    noise_ = np.random.normal(0, 1, (batch_size, nz))
    if not opt.marygan:
        class_onehot = np.zeros((batch_size, num_classes))
        class_onehot[np.arange(batch_size), label] = 1
        noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]
    save_noise[start:end] = noise_
    noise_ = (torch.from_numpy(noise_))
    noise.data.copy_(noise_.view(batch_size, nz))
    fake = netG(noise).data.cpu().numpy()
    fake = np.floor((fake + 1) * 255/2.0).astype(np.uint8)
    x[start:end] = fake

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


### Generate with class filtering

#### FYI: Load `compnet` first

In [72]:
batch_size = opt.train_batch_size
nz = opt.nz
noise = torch.FloatTensor(opt.train_batch_size, nz)
noise = noise.cuda()
num_classes = 10
klass_picked = 9
comp_img_size = 32
# comp_img_size = opt.imageSize

disc_batch = torch.FloatTensor(opt.train_batch_size, 3, comp_img_size, comp_img_size)
disc_batch = disc_batch.cuda()

# create images
n_used_imgs = 4000
n_gen_imgs = ((n_used_imgs // opt.train_batch_size) + 1) * opt.train_batch_size
x = np.empty((n_gen_imgs,3,opt.imageSize,opt.imageSize), dtype=np.uint8)
batch_in = np.empty((batch_size,3,comp_img_size,comp_img_size))
# create a bunch of GAN images
start = 0
pbar = tqdm_notebook(total=n_gen_imgs)
while not start == n_gen_imgs:
    
#for l in  tqdm(range((n_used_imgs // opt.train_batch_size) + 1),desc='Generating'):
    noise.data.resize_(batch_size, nz).normal_(0, 1)
    #label = np.random.randint(0, num_classes, batch_size)
    if klass_picked is None:
        label = np.random.randint(0, num_classes, batch_size)
    else:
        label = np.ones((batch_size,),dtype=int)*klass_picked
    noise_ = np.random.normal(0, 1, (batch_size, nz))
    if not opt.marygan:
        class_onehot = np.zeros((batch_size, num_classes))
        class_onehot[np.arange(batch_size), label] = 1
        noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]
    noise_ = (torch.from_numpy(noise_))
    noise.data.copy_(noise_.view(batch_size, nz))
    fake = netG(noise).data.cpu().numpy()
    fake = np.floor((fake + 1) * 255/2.0).astype(np.uint8)
    
    for bi in range(fake.shape[0]):
        batch_in[bi] = minimal_trans(np.moveaxis(fake[bi],0,-1))
    disc_batch.data.copy_(torch.from_numpy(batch_in));
    batch_out = compnet(disc_batch).detach().data.cpu()
    class_filt = (np.argmax(batch_out,1) == klass_picked).numpy().astype(bool)
    fake_picked = fake[class_filt]
    end = min(start + fake_picked.shape[0], n_gen_imgs)
    
    x[start:end] = fake_picked[:(end-start)]
    start = end
    pbar.update(fake_picked.shape[0])
pbar.close()

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


### Generate with face confidence filtering

In [885]:
batch_size = opt.train_batch_size
nz = opt.nz
noise = torch.FloatTensor(opt.train_batch_size, nz)
noise = noise.cuda()
num_classes = 10
klass_picked = 8
comp_img_size = 64
# comp_img_size = opt.imageSize

disc_batch = torch.FloatTensor(opt.train_batch_size, 3, comp_img_size, comp_img_size)
disc_batch = disc_batch.cuda()

# create images
n_used_imgs = 100
min_conf = 0.99
n_gen_imgs = ((n_used_imgs // opt.train_batch_size) + 1) * opt.train_batch_size
x = np.empty((n_gen_imgs,3,opt.imageSize,opt.imageSize), dtype=np.uint8)
batch_in = np.empty((batch_size,3,comp_img_size,comp_img_size))
# create a bunch of GAN images
start = 0
pbar = tqdm_notebook(total=n_gen_imgs)
while not start == n_gen_imgs:
    
#for l in  tqdm(range((n_used_imgs // opt.train_batch_size) + 1),desc='Generating'):
    noise.data.resize_(batch_size, nz).normal_(0, 1)
    #label = np.random.randint(0, num_classes, batch_size)
    if klass_picked is None:
        label = np.random.randint(0, num_classes, batch_size)
    else:
        label = np.ones((batch_size,),dtype=int)*klass_picked
    noise_ = np.random.normal(0, 1, (batch_size, nz))
    if not opt.marygan:
        class_onehot = np.zeros((batch_size, num_classes))
        class_onehot[np.arange(batch_size), label] = 1
        noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]
    noise_ = (torch.from_numpy(noise_))
    noise.data.copy_(noise_.view(batch_size, nz))
    fake = netG(noise).data.cpu().numpy()
    fake = np.floor((fake + 1) * 255/2.0).astype(np.uint8)
    
    class_filt = np.zeros(batch_size).astype(bool)
    for bi in range(fake.shape[0]):
        res = detector.detect_faces(np.moveaxis(fake[bi],0,-1))
        if not res:
#         if res and res[0]['confidence'] > min_conf:
            class_filt[bi] = True
        
    fake_picked = fake[class_filt]
    end = min(start + fake_picked.shape[0], n_gen_imgs)
    
    x[start:end] = fake_picked[:(end-start)]
    start = end
    pbar.update(fake_picked.shape[0])
pbar.close()

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


In [778]:
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(x), nrow=int(np.sqrt(n_used_imgs)), padding=0, normalize=True)
plt.imshow(np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

In [15]:
plt.figure()
plt.imshow(np.moveaxis(x[0],0,-1))

FigureCanvasNbAgg()

### Classify as well

In [379]:
opt.netD = opt.netG.replace('netG', 'netD')

In [380]:
if opt.imageSize == 32:
    netD = resnet.Discriminator(opt)
elif opt.imageSize == 64:
    netD = resnet_extra.Discriminator(opt)
elif opt.imageSize == 48:
    netD = resnet_48.Discriminator(opt)
else:
    raise NotImplementedError('A network for imageSize %i is not implemented!' % opt.imageSize)

netD.load_state_dict(torch.load(opt.netD))
netD = netD.cuda()

/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:84: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.conv1.weight.data, 1.)
/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:85: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.conv2.weight.data, 1.)
/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:86: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.bypass_conv.weight.data, np.sqrt(2))
/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/GAN_training/models/resnet.py:42: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.conv1.weight.data, 1.)
/cfarhomes/ily

### Sidenote: Interpolate

In [755]:
first = 1687 // 64 + int((np.sqrt(n_used_imgs)))*(1292 // 64)
#sec = 1839 // 64 + int(np.sqrt(n_used_imgs))*(497 // 64)-1
sec = first+1

Ni = 10
noise_ = np.random.normal(0, 1, (batch_size, nz))
noise_[:Ni] = np.linspace(save_noise[first],save_noise[sec],Ni)

In [765]:

noise_ = (torch.from_numpy(noise_))
noise.data.copy_(noise_.view(batch_size, nz))
interp = netG(noise).data.cpu().numpy()
interp = np.floor((interp + 1) * 255/2.0).astype(np.uint8)


/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


In [766]:
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(interp[:20]), nrow=int(20), padding=2, normalize=True)
plt.imshow(np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [767]:
imageio.imsave('~/marygan.png', np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [758]:
### go beyond the boundar

In [764]:
noise_ = noise_.numpy()
d = (save_noise[sec]-save_noise[first]) / 10.0
further1 = save_noise[first] + (d * np.expand_dims(np.arange(20),1))
noise_[:20] = further1

In [728]:
further1.min()

-6.551881954370094

### Load feature extracting network

In [32]:
device = torch.device("cuda:0")

In [772]:
torch.cuda.empty_cache()

In [280]:
from classification.models.vgg5 import VGG
compnet = VGG('VGG19')
compnet = torch.nn.DataParallel(compnet)
checkpoint = torch.load(os.path.join('/scratch0/ilya/locDoc/classifiers/vgg16','ckpt_200.t7'))
compnet.load_state_dict(checkpoint['net'])
compnet = compnet.to(device)
compnet.eval();
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [33]:
# cifar,32
from classification.models.densenet import DenseNet121
compnet = DenseNet121()
compnet = torch.nn.DataParallel(compnet)
checkpoint = torch.load(os.path.join('/scratch0/ilya/locDoc/classifiers/densenet121','ckpt_47.t7'))
compnet.load_state_dict(checkpoint['net'])
compnet = compnet.to(device)
compnet.eval();
transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
minimal_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [551]:
from classification.models.vgg_official2 import vgg16
compnet = vgg16(pretrained=True)
compnet = compnet.to(device)
compnet.eval()
transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [353]:
from classification.models.vgg_face_dag import vgg_face_dag
compnet = vgg_face_dag(weights_path='/scratch0/ilya/locDownloads/vgg_face_dag.pth')
compnet = compnet.to(device)
compnet.eval()
transform_test=transforms.Compose([
    transforms.ToPILImage(),
                           transforms.Resize((224,224)),
                           transforms.Lambda(lambda img: np.moveaxis(np.array(img),-1,0) ),
                           ])

In [567]:
# stl, 64
from classification.models.densenet import densenet_stl as DenseNet121
compnet = DenseNet121()
compnet = torch.nn.DataParallel(compnet)
checkpoint = torch.load(os.path.join('/scratch0/ilya/locDoc/classifiers/stl/densenet121_64/ckpt.t7'))
compnet.load_state_dict(checkpoint['net'])
compnet = compnet.to(device)
compnet.eval();
transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
minimal_trans = transform_test

### Pass imgs through net

In [118]:
#net_in = np.empty((x.shape[0],3,64,64)) # stl
net_in = np.empty((x.shape[0],3,32,32))
#net_in = np.empty((x.shape[0],) + (3,224,224)) # celebA, vgg-official
for i in tqdm_notebook(range(x.shape[0]),desc='Preprocess'):
    net_in[i] = transform_test(np.moveaxis(x[i],0,-1))

my_dataset = utils.TensorDataset(torch.FloatTensor(net_in))
my_dataloader = utils.DataLoader(my_dataset, batch_size=opt.train_batch_size, shuffle=False)

In [119]:
class_out = np.empty(x.shape[0])
logit_out = np.empty((x.shape[0],10))
#net_out = np.empty((x.shape[0], 602112))
#net_out = np.empty((x.shape[0], 12288))
net_out = np.empty((x.shape[0], 1024)) # Densenet
#net_out = np.empty((x.shape[0], 2622)) # vgg-face
#net_out = np.empty((x.shape[0], 25088)) # vgg official penultimate
for i, batch in enumerate(tqdm_notebook(my_dataloader,desc='Extract Feat')):
    start = i * opt.train_batch_size
    end = start + opt.train_batch_size
    batch_in = batch[0].to(device)
    #batch_out = compnet(batch_in).detach().data.cpu() # faces
    batch_out = compnet(batch_in, feat=1).detach().data.cpu() # vgg official
    net_out[start:end] = batch_out
    logit_out[start:end] = compnet(batch_in).detach().data.cpu()
    class_out[start:end] = logit_out[start:end].argmax(1)
    
    

In [73]:
### OR... MSE style
mse_out = x.reshape(x.shape[0], np.prod(x.shape[1:]))

### Face detector

In [7]:
from mtcnn.mtcnn import MTCNN
import cv2

In [8]:
detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [9]:
img = cv2.imread('/cfarhomes/ilyak/Screenshot from 2019-05-19 18-05-24.png')

In [10]:
res = detector.detect_faces(img)

InvalidImage: Image not valid.

In [37]:
detector.detect_faces(np.moveaxis(x[488],0,-1))

[{'box': [1, -4, 60, 71],
  'confidence': 0.9998058676719666,
  'keypoints': {'left_eye': (17, 25),
   'right_eye': (46, 24),
   'nose': (30, 39),
   'mouth_left': (20, 54),
   'mouth_right': (45, 53)}}]

### Plot closest pairs

In [143]:
D1 = pairwise_distances(net_out)
D2 = pairwise_distances(mse_out)

In [144]:
D1 = (D1 - D1.mean()) / D1.std()
D2 = (D2 - D2.mean()) / D2.std()
D = D1+ 0.25*D2
deleted_flag = D.max()

In [80]:
### OR just net_out
D = pairwise_distances(net_out)

In [74]:
### OR just net_out
D = pairwise_distances(mse_out)

In [81]:
# remove the diagonal and lower triangle
to_del = np.tril(np.ones((D.shape[0], D.shape[0]), dtype=int))
D[to_del == 1] = D.max()


dists = D.flatten()
closest_N = 40
idxs = np.argpartition(dists,closest_N)
min_idxs = sorted(idxs[:closest_N], key=lambda i: dists[i])
#print(dists[min_idxs])
closest_idxs = [(idx // D.shape[0], idx % D.shape[0]) for idx in min_idxs]

In [82]:
closest_imgs = np.zeros((closest_N * 2,)+x.shape[1:])
used_idxs = []
l = 0
for (i,j) in closest_idxs:
    #res1 = detector.detect_faces(np.moveaxis(x[i],0,-1))
    #res2 = detector.detect_faces(np.moveaxis(x[j],0,-1))
    if (i not in used_idxs) and (j not in used_idxs):# and res1 and res2:
        closest_imgs[2*l] = x[min(i,j)]
        closest_imgs[2*l + 1] = x[max(i,j)]
        used_idxs.append(i)
        used_idxs.append(j)
        l += 1

In [83]:
# plot closest pairs
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(closest_imgs[:20]), nrow=2, padding=0, normalize=True)
plt.imshow(np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

FigureCanvasNbAgg()

In [55]:
imageio.imsave('~/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/test.png', np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [34]:
used_idxs
#closest_idxs

[460, 550, 710, 846, 222, 1441, 629, 1720, 488, 1652]

In [189]:
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(closest_imgs[20:]), nrow=2, padding=0, normalize=True)
plt.imshow(np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

FigureCanvasNbAgg()

### Save selections manually

In [191]:
row1 = []
row2 = []

In [236]:
aidx = 8
row1.append(closest_imgs[2*aidx])
row2.append(closest_imgs[2*aidx + 1])

In [237]:
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(row1 + row2), nrow=len(row1), padding=0, normalize=True)
fake_grid_img = np.moveaxis(fake_grid.data.cpu().numpy(),0,-1)
plt.imshow(fake_grid_img)

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [238]:
imageio.imsave('~/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/celebA_vanilla_10.png', fake_grid_img)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


### Load diversity statistics that a script output

In [337]:
fpath = '/scratch0/ilya/locDoc/ACGAN/experiments/diversity_plots/'

In [340]:
fpath = '/fs/vulcan-scratch/ilyak/locDoc/experiments/diversity_plots/'

In [341]:
fpath += 'celeba/'

In [342]:
vanilla_data = np.load(fpath + 'vanilla5.npz')
marygan_data = np.load(fpath + 'marygan5.npz')
acgan_data = np.load(fpath + 'acgan5.npz')

FileNotFoundError: [Errno 2] No such file or directory: '/fs/vulcan-scratch/ilyak/locDoc/experiments/diversity_plots/celeba/vanilla5.npz'

In [174]:
t = np.arange(0, vanilla_data['mins'].shape[1]*128, 128)

In [175]:
plt.figure()
def show(color, data):
    y = data.mean(0)
    err = data.std(0)
    plt.plot(t, y, 'k-', color=color)
    plt.fill_between(t, y-err, y+err, facecolor=color, alpha=0.3)
    
show('orange', marygan_data['mins'])
show('green', acgan_data['mins'])
show('blue', vanilla_data['mins'])

plt.legend(['marygan 5 class', 'acgan 5 class', 'vanilla'])
plt.ylabel('Min VGG-Face Embedding distance')
plt.xlabel('n generated examples')
plt.title('GANs on CelebA')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'GANs on CelebA')

### Face Confidence plots

In [5]:
xp_rama = '/scratch0/ilya/locDoc/ACGAN/experiments/'
datas = []
for mode in ['marygan', 'acgan', 'vanilla']:
    data = np.load(xp_rama+'confidence_plots/%s_genW_10001_trial_0.npz' % mode)['confs']
    for fidx in range(1,10):
        data = np.concatenate([data, np.load(xp_rama+'confidence_plots/%s_genW_10001_trial_%i.npz' % (mode, fidx) )['confs']])
    datas.append(data)
    

FileNotFoundError: [Errno 2] No such file or directory: '/scratch0/ilya/locDoc/ACGAN/experiments/confidence_plots/marygan_genW_10001_trial_0.npz'

In [27]:
fig = plt.figure()
ax = fig.add_subplot(111)

FigureCanvasNbAgg()

In [28]:
mybins = np.array([0, 0.9, 0.99,0.999,0.9999])
x_pos = np.arange(len(mybins))
offset = [-0.3, 0, 0.3]
colors = ['green', 'orange', 'blue']
for i, d in enumerate(datas):
    bt = (np.expand_dims(d,-1) > np.expand_dims(mybins,0)).mean(axis=0)
    ax.bar(x_pos + offset[i], bt, 0.3, color=colors[i])

In [21]:
ax.legend(['Mode GAN', 'ACGAN', 'SN GAN'])
ax.axes.set_xticklabels([''] + ['> %.4f' % mybins[i] for i in range(len(mybins))])
ax.set_xlabel('MTCNN face detection confidence')
ax.set_ylabel('Percent of Generator Outputs')

Text(67.31944444444443, 0.5, 'Percent of Generator Outputs')

In [22]:
ax.axes.set_yticks(np.arange(0,1,0.05));
labels = [(('%.2f' % t) if (i % 2 == 0) else '') for (i,t) in enumerate(np.arange(0,1,0.05))]
ax.axes.set_yticklabels(labels);
plt.grid(True)

In [16]:
plt.savefig('/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/celeba_confidence.png')

### Face sorting by score batch plots

In [6]:
x_imgs = x[:100]
confs = np.zeros(100)
pamt = 8
for bi in range(100):
    img = np.pad(np.moveaxis(x[bi],0,-1), ((pamt,pamt),(pamt,pamt),(0,0)), 'edge')
    res = detector.detect_faces(img)
    if res:
        confs[bi] = res[0]['confidence']

NameError: name 'x' is not defined

In [309]:
conf_order = np.argsort(-confs)
conf_square = confs[conf_order].reshape((10,10))

In [310]:
x_sorted = x_imgs[conf_order]

In [311]:
fig = plt.figure()
ax = fig.add_subplot(111)

FigureCanvasNbAgg()

In [312]:
fake_grid = vutils.make_grid(torch.Tensor(x_sorted), nrow=10, padding=0, normalize=True)
plt.imshow(np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

In [313]:
ax.axes.set_xticks(32 + np.arange(0,10) * 64);


In [314]:
ax.axes.set_yticks(32 + np.arange(0,10) * 64);

In [315]:
ax.axes.set_xticklabels(['%.3f' % c for c in confs[conf_order][90:]], rotation=-45);
ax.axes.set_yticklabels([('%.10f' % c)[:6] for c in conf_square[:,0]]);

In [316]:
plt.savefig('/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/celeba_maryg_batch.png')

### Worst face plots

In [886]:
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(x), nrow=int(np.sqrt(n_used_imgs)), padding=0, normalize=True)
plt.imshow(np.moveaxis(fake_grid.data.cpu().numpy(),0,-1))

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

In [890]:
selection = [120,58,47]
print(len(selection))
x[selection].shape

3


(3, 3, 64, 64)

In [894]:
plt.figure()

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

<Figure size 640x480 with 0 Axes>

In [895]:
fake_grid = vutils.make_grid(torch.Tensor(x[selection]), nrow=10, padding=0, normalize=True)
fake_img = np.moveaxis(fake_grid.data.cpu().numpy(),0,-1)
plt.imshow(fake_img)

In [893]:
imageio.imsave('/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/celeba_marygan_stretch_bad_eg2.png', fake_img)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [812]:
imageio.imsave('/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/celeba_celeba_bad_eg.png', fake_img)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


### TSNE embeddings

In [343]:
from sklearn.manifold import TSNE

In [431]:
tsne_fitter = TSNE()

In [538]:
tsne_embed = tsne_fitter.fit_transform(net_out)

In [539]:
tx, ty = tsne_embed[:,0], tsne_embed[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [540]:
fig = plt.figure()
ax = fig.add_subplot(111)

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

In [541]:
plt.scatter(tx,ty,c=class_out.astype(int), alpha=0.4,  cmap='jet')

In [542]:
from PIL import Image
width = 4000
height = 3000
max_dim = 100.
full_image = Image.new('RGB', (width, height))
for idx in range(x.shape[0]):
    tile = Image.fromarray(np.moveaxis(x[idx],0,-1))
    rs = max(1, tile.width / max_dim, tile.height / max_dim)
    tile = tile.resize((int(tile.width / rs),
                        int(tile.height / rs)),
                       Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim) * tx[idx]),
                            int((height-max_dim) * ty[idx])))

In [543]:
plt.imshow(full_image)

In [544]:
imageio.imsave('/scratch0/ilya/locDownloads/stl_tsne.png', full_image)

In [505]:
plt.figure()

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

<Figure size 640x480 with 0 Axes>

### Batches sorted by one of the 10 classes

In [120]:
conf_out = np.abs(np.expand_dims(logit_out.max(1),-1) - logit_out).sum(1)
ord_idx = np.argsort(-conf_out)

In [121]:
# or just natural order
ord_idx = np.arange(0,x.shape[0],dtype=int)

In [122]:
width = 30

In [123]:
sorted_fakes = np.zeros((10,width,3,x.shape[-1],x.shape[-1]))
idx_p = np.zeros(width,dtype=int)
for idx in ord_idx:
    lab = int(class_out[idx])
    insert_idx = idx_p[lab]
    if insert_idx < width:
        sorted_fakes[lab,insert_idx,:,:,:] = x[idx]
        idx_p[lab] += 1

In [124]:
plt.figure()
fake_grid = vutils.make_grid(torch.Tensor(sorted_fakes.reshape((10*width,3,x.shape[-1],x.shape[-1]))), nrow=width, padding=0, normalize=True)
fake_img = np.moveaxis(fake_grid.data.cpu().numpy(),0,-1)
plt.imshow(fake_img)

FigureCanvasNbAgg()

In [125]:
imageio.imsave('/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/ACGAN-PyTorch/figs/cifar_vanilla_wide_batch.png', fake_img)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


### Probabilities (CIFAR)

In [769]:
fig = plt.figure()
ax = fig.add_subplot(111)

/scratch0/ilya/locDoc/miniconda2/envs/venvcuda10/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

In [749]:
datas = []

In [766]:
probs_out = (np.exp(logit_out.max(1)) / np.exp(logit_out).sum(1))

In [767]:
datas.append(probs_out)

In [720]:
hist, binso = np.histogram(probs_out, bins=20)

In [721]:
ax.plot(hist)

In [722]:
probs_out.max()

0.9999993230708444

In [770]:
mybins = np.array([0, 0.9, 0.99,0.999,0.9999])
x_pos = np.arange(len(mybins))
offset = [-0.3, 0, 0.3]
for i, d in enumerate(datas):
    bt = (np.expand_dims(d,-1) > np.expand_dims(mybins,0)).mean(axis=0)
    ax.bar(x_pos + offset[i], bt, 0.3)